In [1]:
import math
import pandas as pd
import numpy as np

In [2]:
#documents
doc1 = "I want to start learning to charge something in life"
doc2 = "reading something about life no one else knows"
doc3 = "Never stop learning"
#query string
query = "life learning"

In [3]:
#finding term frequency from documents
#term -frequenvy :word occurences in a document

def compute_tf(docs_list):
    for doc in docs_list: 
        doc1_lst = doc.split(" ")
        wordDict_1= dict.fromkeys(set(doc1_lst), 0)

        for token in doc1_lst:
            wordDict_1[token] +=  1
        df = pd.DataFrame([wordDict_1])
        idx = 0
        new_col = ["Term Frequency"]    #adding new column for term frequency count
        df.insert(loc=idx, column='Document', value=new_col) #adding new column for document name
        print(df)
        
compute_tf([doc1, doc2, doc3])

         Document  want  in  life  learning  start  to  charge  something  I
0  Term Frequency     1   1     1         1      1   2       1          1  1
         Document  one  no  life  about  else  reading  something  knows
0  Term Frequency    1   1     1      1     1        1          1      1
         Document  Never  learning  stop
0  Term Frequency      1         1     1


In [4]:
#Normalized Term Frequency
def termFrequency(term, document):
    normalizeDocument = document.lower().split() #normalizing using lowering the words
    return normalizeDocument.count(term.lower()) / float(len(normalizeDocument))

def compute_normalizedtf(documents):
    tf_doc = []
    for txt in documents:
        sentence = txt.split()
        norm_tf= dict.fromkeys(set(sentence), 0)
        for word in sentence:
            norm_tf[word] = termFrequency(word, txt)
        tf_doc.append(norm_tf)
        df = pd.DataFrame([norm_tf])
        idx = 0
        new_col = ["Normalized TF"]    #adding new col for normalized term frequency count
        df.insert(loc=idx, column='Document', value=new_col)
        print(df)
    return tf_doc

tf_doc = compute_normalizedtf([doc1, doc2, doc3])

        Document  want   in  life  learning  start   to  charge  something  \
0  Normalized TF   0.1  0.1   0.1       0.1    0.1  0.2     0.1        0.1   

     I  
0  0.1  
        Document    one     no   life  about   else  reading  something  knows
0  Normalized TF  0.125  0.125  0.125  0.125  0.125    0.125      0.125  0.125
        Document     Never  learning      stop
0  Normalized TF  0.333333  0.333333  0.333333


In [5]:
#Inverse Document Frequency
def inverseDocumentFrequency(term, allDocuments): #term is the word and allDocuments is the list of documents
    numDocumentsWithThisTerm = 0
    #calculating idf for each word in the documents
    for doc in range (0, len(allDocuments)):
        if term.lower() in allDocuments[doc].lower().split(): #normalizing the term
            numDocumentsWithThisTerm = numDocumentsWithThisTerm + 1
 
    if numDocumentsWithThisTerm > 0: #if the term is not in the document, then the idf is 0
        return 1.0 + math.log(float(len(allDocuments)) / numDocumentsWithThisTerm)
    else:
        return 1.0
    
def compute_idf(documents): #documents is the list of documents
    idf_dict = {} #dictionary to store the idf values
    for doc in documents:
        sentence = doc.split()
        for word in sentence:
            idf_dict[word] = inverseDocumentFrequency(word, documents) #calling the inverseDocumentFrequency function
    return idf_dict
idf_dict = compute_idf([doc1, doc2, doc3])

compute_idf([doc1, doc2, doc3]) #computing idf for each word in the documents

{'I': 2.09861228866811,
 'want': 2.09861228866811,
 'to': 2.09861228866811,
 'start': 2.09861228866811,
 'learning': 1.4054651081081644,
 'charge': 2.09861228866811,
 'something': 1.4054651081081644,
 'in': 2.09861228866811,
 'life': 1.4054651081081644,
 'reading': 2.09861228866811,
 'about': 2.09861228866811,
 'no': 2.09861228866811,
 'one': 2.09861228866811,
 'else': 2.09861228866811,
 'knows': 2.09861228866811,
 'Never': 2.09861228866811,
 'stop': 2.09861228866811}

In [6]:
#calculating tf*idf
# tf-idf score across all docs for the query string("life learning")
def compute_tfidf_with_alldocs(documents , query): #documents is the list of documents and query is the query string
    tf_idf = [] #list  to store the tf-idf values
    index = 0
    query_tokens = query.split()
    df = pd.DataFrame(columns=['doc'] + query_tokens)
    for doc in documents:
        df['doc'] = np.arange(0 , len(documents))
        doc_num = tf_doc[index]
        sentence = doc.split()
        for word in sentence: #calculating tf-idf for each word in the documents
            for text in query_tokens: #calculating tf-idf for each word in the query string
                if(text == word): #if the word in the query string is in the document, then calculate tf-idf
                    idx = sentence.index(word)
                    tf_idf_score = doc_num[word] * idf_dict[word]
                    tf_idf.append(tf_idf_score)
                    df.iloc[index, df.columns.get_loc(word)] = tf_idf_score
        index += 1
    df.fillna(0 , axis=1, inplace=True)
    return tf_idf , df
            
documents = [doc1, doc2, doc3] #documents
tf_idf , df = compute_tfidf_with_alldocs(documents , query) #calling the compute_tfidf_with_alldocs function
print(df)

   doc      life  learning
0  0.0  0.140547  0.140547
1  1.0  0.175683  0.000000
2  2.0  0.000000  0.468488


In [7]:
#Normalized TF for the query string
def compute_query_tf(query):
    query_norm_tf = {}
    tokens = query.split()
    for word in tokens:
        query_norm_tf[word] = termFrequency(word , query)
    return query_norm_tf
query_norm_tf = compute_query_tf(query)
print(query_norm_tf)

{'life': 0.5, 'learning': 0.5}


In [8]:
#idf score for the query string
def compute_query_idf(query): #computes the idf score for the query string
    idf_dict_qry = {}
    sentence = query.split()
    documents = [doc1, doc2, doc3]
    for word in sentence:
        idf_dict_qry[word] = inverseDocumentFrequency(word ,documents)
    return idf_dict_qry
idf_dict_qry = compute_query_idf(query)
print(idf_dict_qry)

{'life': 1.4054651081081644, 'learning': 1.4054651081081644}


In [9]:
#tf-idf score for the query string
def compute_query_tfidf(query): #this function calculates the tf-idf score for the query string
    tfidf_dict_qry = {}
    sentence = query.split()
    for word in sentence:
        tfidf_dict_qry[word] = query_norm_tf[word] * idf_dict_qry[word]
    return tfidf_dict_qry
tfidf_dict_qry = compute_query_tfidf(query)
print(tfidf_dict_qry)

{'life': 0.7027325540540822, 'learning': 0.7027325540540822}


In [10]:
#Cosine Similarity(Query,Document1) = Dot product(Query, Document1) / ||Query|| * ||Document1||

"""
Example : Dot roduct(Query, Document1) 

     life:
     = tfidf(life w.r.t query) * tfidf(life w.r.t Document1) +  / 
     sqrt(tfidf(life w.r.t query)) * 
     sqrt(tfidf(life w.r.t doc1))
     
     learning:
     =tfidf(learning w.r.t query) * tfidf(learning w.r.t Document1)/
     sqrt(tfidf(learning w.r.t query)) * 
     sqrt(tfidf(learning w.r.t doc1))

"""
#this function calculates the cosine similarity between the query string and the documents
def cosine_similarity(tfidf_dict_qry, df , query , doc_num):
    dot_product = 0
    qry_mod = 0
    doc_mod = 0
    tokens = query.split()
   
    for keyword in tokens:
        dot_product += tfidf_dict_qry[keyword] * df[keyword][df['doc'] == doc_num]
        #||Query||
        qry_mod += tfidf_dict_qry[keyword] * tfidf_dict_qry[keyword]
        #||Document||
        doc_mod += df[keyword][df['doc'] == doc_num] * df[keyword][df['doc'] == doc_num]
    qry_mod = np.sqrt(qry_mod)
    doc_mod = np.sqrt(doc_mod)
    #implement formula
    denominator = qry_mod * doc_mod
    cos_sim = dot_product/denominator
     
    return cos_sim

from collections import Iterable
def flatten(lis):
     for item in lis:
        if isinstance(item, Iterable) and not isinstance(item, str):
             for x in flatten(item):
                yield x
        else:        
             yield item

C:\Users\nusan\AppData\Local\Temp\ipykernel_17856\2873275391.py:37: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable


In [11]:
def rank_similarity_docs(data): #data is a dataframe
    cos_sim =[] # to store the cosine similarity values
    for doc_num in range(0 , len(data)):
        cos_sim.append(cosine_similarity(tfidf_dict_qry, df , query , doc_num).tolist()) #calling the cosine_similarity function to store the cosine similarity values
    return cos_sim
similarity_docs = rank_similarity_docs(documents) #calling the rank_similarity_docs function
doc_names = ["Document1", "Document2", "Document3"] #document names
print(doc_names) 
print(list(flatten(similarity_docs))) #calling the flatten function to flatten the list

['Document1', 'Document2', 'Document3']
[1.0, 0.7071067811865475, 0.7071067811865475]
